# NASDAQ

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
# gráficos interactivos
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#import chart_studio.plotly as py
# módulos optimización
import scipy.optimize as optimize
import cvxopt as opt
from cvxopt import matrix, spmatrix, sparse
from cvxopt.solvers import qp, options
from cvxopt import blas
# web scrapping
import bs4 as bs
import requests
import lxml
from functools import reduce
# matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from ipysigma import Sigma
from pyvis.network import Network
import requests
from bs4 import BeautifulSoup
from io import StringIO
from dbconnection import MySQLDatabase
sns.set()

In [3]:
db = MySQLDatabase("financialmarkets")

In [4]:
year = '2024'

In [5]:
query = f"""   
    SELECT 
        sp.date,
        c.symbol,
        sp.close_price
    FROM stock_prices_nasdaq sp
    JOIN companies c ON sp.company_id = c.company_id
    WHERE YEAR(sp.date) = {2025};
"""
df = db.execute_query(query)
print(f"Shape: {df.shape}")

✅ Conexión exitosa
Shape: (1275149, 3)


In [6]:
base = df.pivot_table(index='date', columns=['symbol'], values='close_price').reset_index()
base.head()

symbol,date,AACB,AACBR,AACBU,AACG,AACI,AACIU,AACIW,AADR,AAL,...,ZUMZ,ZURA,ZVRA,ZVZZT,ZWZZT,ZXZZT,ZYBT,ZYME,ZYN,ZYXI
0,2025-01-01,NaN,NaN,NaN,0.865,NaN,NaN,NaN,69.72,17.215,...,19.07,2.425,8.55,NaN,NaN,NaN,NaN,14.71,NaN,7.91
1,2025-01-02,NaN,NaN,NaN,0.88,NaN,NaN,NaN,70.42,17.0,...,18.97,2.35,8.76,NaN,NaN,NaN,NaN,14.78,NaN,7.81
2,2025-01-03,NaN,NaN,NaN,0.9,NaN,NaN,NaN,70.94,16.97,...,19.17,2.58,8.59,NaN,NaN,NaN,NaN,15.01,NaN,8.09
3,2025-01-04,NaN,NaN,NaN,0.9133,NaN,NaN,NaN,71.0533,17.1533,...,19.11,2.5533,8.54,NaN,NaN,NaN,NaN,14.8267,NaN,8.0867
4,2025-01-05,NaN,NaN,NaN,0.9267,NaN,NaN,NaN,71.1667,17.3367,...,19.05,2.5267,8.49,NaN,NaN,NaN,NaN,14.6433,NaN,8.0833


In [7]:
base = base.dropna(axis=1).set_index('date')

In [8]:
# Calculate daily returns
returns = base.pct_change().dropna()

# Compute correlation matrix
correlation_matrix = returns.corr()

C:\Users\HP\AppData\Local\Temp\ipykernel_18132\3453918313.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  returns = base.pct_change().dropna()


In [ ]:
# Crear heatmap
# plt.figure(figsize=(8,6))
# sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm")
# plt.title("Heatmap de la matriz de correlaciones")
# plt.show()

In [11]:
# Set correlation threshold
threshold = 0.80

# Create graph
G = nx.Graph()

# Add nodes
for ticker in correlation_matrix.columns:
    G.add_node(ticker)

# Add edges with correlation above threshold
for i, stock1 in enumerate(correlation_matrix.columns):
    for j, stock2 in enumerate(correlation_matrix.columns):
        if i < j:
            corr = correlation_matrix.iloc[i, j]
            if abs(corr) >= threshold:
                G.add_edge(stock1, stock2, weight=corr)


Sigma.write_html(
    G,
    f"./grafos/nasdaq_{year}.html",
    fullscreen=True,
    node_metrics=['louvain'],
    node_color='louvain',
    node_size_range=(3, 15),
    max_categorical_colors=30,
    default_edge_type='curve',
    node_border_color_from='node',
    default_node_label_size=14,
    node_size=G.degree
)